In [106]:
import cv2 as cv
import imutils
import numpy as np

In [107]:
def LPD(img,padding=50):
    #? Step 1: Edge Detection
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = gray[gray.shape[0]//2:gray.shape[0],:]
    gray = cv.GaussianBlur(gray, (3,3), 1)
    rectKern = cv.getStructuringElement(cv.MORPH_RECT, (7,5))
    # The Black Hat operation is the difference between the closing and input image 
    blackhat = cv.morphologyEx(gray, cv.MORPH_BLACKHAT, rectKern)
    #cv.imshow("blackhat", blackhat)
    
    sobelx = cv.Sobel(blackhat, cv.CV_64F, 1, 0, ksize=3)

    # Calculate the gradient magnitude

    sobelx = np.absolute(sobelx)
    maxVal = np.max(sobelx)
    sobelx = 255 * ((sobelx) / (maxVal))
    sobelx = sobelx.astype("uint8")
    #cv.imshow("grad", sobelx)
    
    
    closeKern = cv.getStructuringElement(cv.MORPH_RECT, (11,11))
    
    closed_image = cv.morphologyEx(sobelx, cv.MORPH_CLOSE, closeKern)
    #cv.imshow("closed", closed_image)
    thresh = cv.threshold(closed_image, 0, 255,cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
    #cv.imshow("thresh", thresh)
    
    Eroded_img = cv.erode(thresh, (3,3), iterations=2)
    #cv.imshow("eroded", Eroded_img)
    Dilated_img = cv.dilate(Eroded_img, (3,3), iterations=3)
    #cv.imshow("dilated", Dilated_img)
    
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv.contourArea, reverse=True)

    filtered_cnts = []
    roi = None

    for countour in cnts: 
        area = cv.contourArea(countour)
        (x, y, w, h) = cv.boundingRect(countour)
        ar = w / float(h)
        if (ar>=2.5 and ar<=5):
            filtered_cnts.append(countour)
            licensePlate = gray[y:y + h + padding, x:x + w + padding]
            roi = cv.threshold(licensePlate, 0, 255,cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]

            cv.imshow("License Plate", licensePlate)
            # cv.imshow("ROI", roi)
            break

        
        
    # cv.drawContours(img[img.shape[0]//2:img.shape[0],:], filtered_cnts, -1, (0,255,0), 3)
    cv.imshow("contours", img)  
    
    
    
    

In [108]:
LPD(cv.imread("0081.jpg"))
cv.waitKey(0)

-1